In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import glob
import os
from ripple_heterogeneity.utils import functions

functions.set_plotting_defaults()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
fig_save_path = r'Z:\home\ryanh\projects\ripple_heterogeneity\figures\panels'

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [7]:
files[2]

'Z:/home/Heath/Viral_tracing/CTB\\CTBR_anlaysis\\CTBR1\\day1\\experiment_17\\experiment_17_greencells.csv'

In [8]:
files = glob.glob(r"Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis/**/*.csv",recursive=True)
# rats = [""]
df = pd.DataFrame()
for file in files:
    df_temp = pd.read_csv(file)
    df_temp["filename"] = file
    df = pd.concat([df,df_temp],ignore_index=True)

df

,,Area,Mean,Min,Max,X,Y,Ch,filename,Slice
0,1,0.0,4774,4774,4774,141.007,115.426,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,NaN
1,2,0.0,3664,3664,3664,143.191,50.226,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,NaN
2,1,0.0,4615,4615,4615,236.468,99.828,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,1.0
3,2,0.0,2216,2216,2216,110.747,55.841,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,1.0
4,1,0.0,5642,5642,5642,136.640,99.204,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,13.0
...,...,...,...,...,...,...,...,...,...,...
1258,51,0.0,34670,34670,34670,58.140,233.198,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,8.0
1259,52,0.0,12931,12931,12931,446.591,207.643,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,7.0
1260,53,0.0,18194,18194,18194,79.862,222.976,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,7.0
1261,54,0.0,10577,10577,10577,102.863,208.920,1.0,Z:/home/Heath/Viral_tracing/CTB/CTBR_anlaysis\...,7.0
